In [ ]:
 from google.colab import drive
drive.mount('/content/drive')
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!git clone https://github.com/jasonkyuyim/se3_diffusion

--2024-04-18 05:22:28--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143351488 (137M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 136.71M   189MB/s    in 0.7s    

2024-04-18 05:22:29 (189 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [143351488/143351488]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPA

In [ ]:
%cd ./se3_diffusion
!conda env create -f se3.yml

In [ ]:
%%bash
source activate se3
pip install -e ./

Obtaining file:///content/se3_diffusion
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for se3-diffusion


In [ ]:
#God knows why, but this suddenly started having cuda problems. Seems to be fixed by uninstalling and reinstalling torch packages
%%bash
source activate se3
conda uninstall pytorch torchvision torchaudio pytorch-cuda

In [ ]:
%%bash
source activate se3
conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.6 -c pytorch -c nvidia

In [ ]:
import os
import shutil
import glob
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uuid
from datetime import datetime
import re
import torch
from time import time

meta_data_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/framediff_unconditional/generation_metadata_framediff.csv"

if os.path.exists(meta_data_filepath):
  all_metadata_df = pd.read_csv(meta_data_filepath)
  print("Existing generation metadata read in.")
else:
  all_metadata_df = pd.DataFrame()
  #all_metadata_df.to_csv(meta_data_filepath, index=False)
  print("Created generation metadata dataframe")


len_dist_filepath = "/content/drive/MyDrive/Generative_Models/unconditional_generation/framediff_unconditional/uniref50_length_dist_framediff.json"

if os.path.exists(len_dist_filepath):
  with open(len_dist_filepath, "r") as f:
    uniprot_length_dist =  json.load(f)
  print("Loaded length distribution from drive")
else:

  #https://www.uniprot.org/uniprotkb/statistics#sequence-size
  bins = np.array([13,51,101,151,201,251,301,351,401,451,501,551,601,651,701,751,801,851,901,951,1001,1101,1201,1301,1401,1501,1601,1701,1801,1901,2001,2101,2201,2301,2401,2501,34350])
  swissprot_reviewed = np.array([0,9968,43534,59796,59574,58452,52413,52846,45901,37706,30572,22287,15830,13156,9403,7870,5700,4889,5301,4109,3007,4124,2897,2207,2070,1675,834,642,587,503,395,272,386,340,234,195,1462])
  TrEMBL_unreviewed = np.array([0,2668805,19825275,24705701,23838128,23462438,23225451,21389271,16814580,14287105,11501843,8283150,6266068,4715059,3755005,3186452,2687314,2166878,1843669,1457871,1153537,1975953,1398765,961048,664766,517536,390552,300984,236895,210921,180246,138808,122833,102865,82441,71548,527646])

  ecdf = np.cumsum(swissprot_reviewed) / np.sum(swissprot_reviewed)
  #shortest protein in uniprot is 14 res, longest is 34350 res.
  x = np.arange(14, 34350+1)
  ecdf = np.interp(x, bins, ecdf)

  # Sample from the empirical CDF
  num_samples = 11000
  random_values = np.random.rand(num_samples)
  sampled_lengths = np.round(np.interp(random_values, ecdf, x)).astype(int)
  #ten thousand sequences up to 1000 res in length
  sampled_lengths = sampled_lengths[sampled_lengths <= 1000][0:10000]

  # Plot the histogram of sampled values
  hist_values, bin_edges, patches = plt.hist(sampled_lengths, bins=x[0:1001-13], alpha=0.7, label='Sampled Values')
  plt.xlabel('X-axis label')
  plt.ylabel('Frequency')
  plt.legend()
  plt.show()

  uniprot_length_dist = list(zip([int(edge) for edge in bin_edges],[int(value) for value in hist_values]))
  with open(len_dist_filepath, "w") as f:
      json.dump(uniprot_length_dist, f)


Mounted at /content/drive
Existing generation metadata read in.
Loaded length distribution from drive


In [ ]:
#because of the code structure we need to first save the metadata to a local copy and then transfer back
all_metadata_df.to_csv("./generation_metadata_framediff.csv",index=False)

In [ ]:
generation_command = """

source activate se3
python -c '

import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

import os
import shutil
import uuid
from datetime import datetime
import time
import tree
import numpy as np
import hydra
import subprocess
import logging
import pandas as pd
from datetime import datetime
from biotite.sequence.io import fasta
import GPUtil
from typing import Optional
import psutil
from analysis import utils as au
from analysis import metrics
from data import utils as du
from data import residue_constants
from typing import Dict
from experiments import train_se3_diffusion
from omegaconf import DictConfig, OmegaConf
from openfold.data import data_transforms
import esm
import experiments.inference_se3_diffusion as se3_inf

@hydra.main(version_base=None, config_path="./config", config_name="inference")
def my_app(conf : DictConfig) -> None:

  local_metadata = "./generation_metadata_framediff.csv"
  try:
    all_metadata_df = pd.read_csv(local_metadata)
  except pd.errors.EmptyDataError:
    all_metadata_df = pd.DataFrame()

  torch.cuda.empty_cache()
  torch.cuda.set_device(0)
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  sampler = se3_inf.Sampler(conf)
  length = conf["inference"]["samples"]["max_length"]
  batch_size = conf["inference"]["samples"]["samples_per_length"]

  meta_data = {}
  meta_data["batch_id"] = None
  meta_data["batch_size"] = None
  meta_data["Timestamp"] = str(datetime.now())
  meta_data["model"] = "FrameDiff"
  meta_data["task"] = "backbone_pdb_generation"
  meta_data["conditions"] = "length = " + str(int(length))
  meta_data["gpu"] = "T4 GPU"
  #adapted from sampler.run_sampling() [excluding the protein mpnn part]
  all_sample_lengths = range(
            sampler._sample_conf.min_length,
            sampler._sample_conf.max_length+1,
            sampler._sample_conf.length_step
        )
  for sample_length in all_sample_lengths:
    print("LENGTH: " + str(sample_length))
    length_dir = os.path.join(
        sampler._output_dir, f"length_{sample_length}")
    os.makedirs(length_dir, exist_ok=True)
    sampler._log.info(f"Sampling length {sample_length}: {length_dir}")
    print(sampler.device)
    for sample_i in range(sampler._sample_conf.samples_per_length):
      sample_dir = os.path.join(length_dir, f"sample_{sample_i}")
      if os.path.isdir(sample_dir):
          continue
      os.makedirs(sample_dir, exist_ok=True)

      start_time = time.time()
      sample_output = sampler.sample(sample_length)
      traj_paths = sampler.save_traj(
          sample_output["prot_traj"],
          sample_output["rigid_0_traj"],
          np.ones(sample_length),
          output_dir=sample_dir
      )
      end_time = time.time()
      total_job_time = end_time - start_time
      meta_data["wall_time_batch"] = None
      meta_data["wall_time_task"] = str(total_job_time) + " Seconds"
      meta_data["entity_id"] = str(uuid.uuid4())
      new_file_name = "FrameDiff_len" + str(length) + "_" + meta_data['entity_id] + ".pdb"
      meta_data["output_file_name"] = new_file_name
      shutil.move(sample_dir +"/sample_1.pdb", "./"+ new_file_name)
      metadata_entry = pd.Series(meta_data)
      all_metadata_df = all_metadata_df.append(metadata_entry, ignore_index=True)

  all_metadata_df.to_csv(local_metadata, index=False)

my_app()
'
"""

In [ ]:
import yaml
with open("./config/inference.yaml", 'r') as file:
        config = yaml.safe_load(file)

In [ ]:
for length, batch_size in uniprot_length_dist:
  if os.path.exists(meta_data_filepath):
    all_metadata_df = pd.read_csv(meta_data_filepath)
  else:
    all_metadata_df = pd.DataFrame()
  if all_metadata_df.loc[all_metadata_df.conditions == "length = " + str(length),:].shape[0] >= batch_size: continue

  config['inference']['samples']['max_length'] = length
  config['inference']['samples']['min_length'] = length
  config['inference']['samples']['name'] = "len_"+str(length)
  config['inference']['samples']['samples_per_length'] = batch_size
  with open("./config/inference.yaml", 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

  !{generation_command}
  print("Metadata saved. Cleaning up....")
  for pdb_file in glob.glob("*.pdb"):
    file_basename = os.path.basename(pdb_file)
    shutil.copy(pdb_file, "/content/drive/MyDrive/Generative_Models/unconditional_generation/framediff_unconditional/" +file_basename)
  shutil.copy("./generation_metadata_framediff.csv",meta_data_filepath)
  for pdb_file in glob.glob("*.pdb"):
    os.remove(pdb_file)
  !rm -rf ./inference_outputs/*
  torch.cuda.empty_cache()
  print("Cleanup complete")